# Peer-to-Peer (P2P) with 2x NVIDIA RTX A5000   

- cloud provider: [runpod.io](https://www.runpod.io/)
- pod: 2 x RTX A5000 (19 vCPU 100 GB RAM)
- image: `runpod/pytorch:2.2.0-py3.10-cuda12.1.1-devel-ubuntu22.04`

### nvidia-smi


In [1]:
!nvidia-smi

Wed Mar 27 20:14:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:56:00.0 Off |                  Off |
| 30%   28C    P8              24W / 230W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvidia-smi topo -m

	GPU0	GPU1	NIC0	NIC1	CPU Affinity	NUMA Affinity	GPU NUMA ID
GPU0	 X 	PIX	SYS	SYS	0-23,48-71	0		N/A
GPU1	PIX	 X 	SYS	SYS	0-23,48-71	0		N/A
NIC0	SYS	SYS	 X 	PIX				
NIC1	SYS	SYS	PIX	 X 				

Legend:

  X    = Self
  SYS  = Connection traversing PCIe as well as the SMP interconnect between NUMA nodes (e.g., QPI/UPI)
  NODE = Connection traversing PCIe as well as the interconnect between PCIe Host Bridges within a NUMA node
  PHB  = Connection traversing PCIe as well as a PCIe Host Bridge (typically the CPU)
  PXB  = Connection traversing multiple PCIe bridges (without traversing the PCIe Host Bridge)
  PIX  = Connection traversing at most a single PCIe bridge
  NV#  = Connection traversing a bonded set of # NVLinks

NIC Legend:

  NIC0: mlx5_0
  NIC1: mlx5_1



### Building [nccl-tests](https://github.com/NVIDIA/nccl-tests/tree/master)

In [3]:
!git clone https://github.com/NVIDIA/nccl-tests.git && cd nccl-tests/ && make

fatal: destination path 'nccl-tests' already exists and is not an empty directory.


### Running all_reduce_perf

In [7]:
!nccl-tests/build/all_reduce_perf -b 8 -e 128M -f 2 -g 2

# nThread 1 nGpus 2 minBytes 8 maxBytes 134217728 step: 2(factor) warmup iters: 5 iters: 20 agg iters: 1 validation: 1 graph: 0
#
# Using devices
#  Rank  0 Group  0 Pid 361891 on 58e5f0f169a6 device  0 [0x56] NVIDIA RTX A5000
#  Rank  1 Group  0 Pid 361891 on 58e5f0f169a6 device  1 [0x57] NVIDIA RTX A5000
#
#                                                              out-of-place                       in-place          
#       size         count      type   redop    root     time   algbw   busbw #wrong     time   algbw   busbw #wrong
#        (B)    (elements)                               (us)  (GB/s)  (GB/s)            (us)  (GB/s)  (GB/s)       
           8             2     float     sum      -1    19.51    0.00    0.00      0    17.81    0.00    0.00      0
          16             4     float     sum      -1    18.03    0.00    0.00      0    17.85    0.00    0.00      0
          32             8     float     sum      -1    17.63    0.00    0.00      0    17.52    0.00    

In [1]:
import torch
import torch.utils.benchmark as benchmark

device0 = torch.device("cuda", 0)
device1 = torch.device("cuda", 1)

x0 = torch.randn(1024, 1024, 1024, dtype=torch.float32, device=device0)
x1 = torch.randint(0,100, (1024, 1024, 1024), dtype=torch.long, device=device1)

def copy_tensor(x, dest_device):
    y = x.to(dest_device, non_blocking=False, copy=False)
    return y

t0 = benchmark.Timer(
    stmt='copy_tensor(x0, device1)',
    setup='from __main__ import copy_tensor',
    globals={'x0': x0, 'device1': device1},
    num_threads=1)

t1 = benchmark.Timer(
    stmt='copy_tensor(x1, device1)',
    setup='from __main__ import copy_tensor',
    globals={'x1': x1, 'device1': device0},
    num_threads=1)

# sanity check
s0 = x0.sum().cpu()
y1 = copy_tensor(x0, device1)
s1 = y1.sum().cpu()
assert torch.abs(s0-s1) < 1e-5

m0 = t0.timeit(100)
storage_size0 = x0.untyped_storage().size()
print(f"{device0}->{device1}: {storage_size0/m0.mean/2**30:.3f} GB/s")

m1 = t1.timeit(100)
storage_size1 = x1.untyped_storage().size()
print(f"{device1}->{device0}: {storage_size1/m1.mean/2**30:.3f} GB/s")

cuda:0->cuda:1: 24.572 GB/s
cuda:1->cuda:0: 24.535 GB/s


In [10]:
!OMP_NUM_THREADS=1 /usr/local/bin/torchrun --standalone --nnodes=1 --nproc-per-node 2 torch_distributed_nccl_test.py

rank=0 send N=50, elapsed_time=8.6825s, 23.035 GB/s, sum=-9481.1240234375 (cuda:0)
rank=1 recv N=50, elapsed_time=8.6485s, 23.125 GB/s, sum=-9481.1240234375 (cuda:1)
rank=0 broadcast(a, src=0) N=50, elapsed_time=9.5473s, 20.948 GB/s, sum=-9481.1240234375 (cuda:0)
rank=0 broadcast(a, src=1) N=50, elapsed_time=9.3507s, 21.389 GB/s, sum=-9481.1240234375 (cuda:0)


In [8]:
torch.__version__

'2.2.0+cu121'